지금까지 보아온 "사과 쇼핑" 예를 파이썬으로 구현한다. 여기에서는 계산 그래프의 곱셈 노드를 "MulLayer", 덧셈 노드를 "AddLaye"라는 이름으로 구현한다.

# 5.4.1 곱셈 계층

모든 계층은 forward()와 backward()라는 공통 메소드를 갖도록 구현할 것이다. forward()는 순전파, backward()는 역전파를 처리한다.

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

"init()"에서는 인스턴스 변수인 x와 y를 초기화 한다. 이 두 변수는 순전파 시의 입력값을 유지하기 위해서 사용한다. forward()에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다. forward()에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다. 반면 backward()에서는 상류에서 넘어온 미분에 순전파 대의 값을 "서로 바꿔" 곱한 후 하류로 흘린다.<br>
이상이 MulLayer의 구현이다. 이 MulLayer를 사용해서 앞에서 본 "사과 쇼핑"을 구현해보자. 앞 절에서는 계산 그래프의 순전파와 역전파를 써서 다음 그림과 같이 계산할 수 있었다.

<img src=images/5_16.png height=100px width=500px>

In [4]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


또 각 변수에 대한 미분은 backward()에서 구할 수 있다.

In [5]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


backward()가 받는 인수는 순전파의 출력에 의한 미분임을 유의하자.

# 5.4.2 덧셈 계층

In [8]:
class AddLayer:
    def __init__(self):
        pass
        
    def forward(self, x, y):
        out = x + y
        
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

덧셈 계층에서는 포기화가 필요 없으니 init()에서는 아무 일도 하지 않는다. 덧셈 계층의 forward()에서는 입력받은 두 인수 x, y를 더해서 반환한다. backward()에서는 상류에서 내려온 미분(dout)을 그대로 하류로 흘릴 뿐이다.
이상의 덧셈 계층과 곱셈 계층을 사용하여 사과 2개와 귤 3개를 사는 다음의 상황을 구현해보자.

<img src=images/5_17.png height=100px width=500px>

In [10]:
apple = 100
apple_num = 2
mandarin = 150
mandarin_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_mandarin_layer = MulLayer()
add_price_layer = AddLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
mandarin_price = mul_mandarin_layer.forward(mandarin, mandarin_num)
total_price = add_price_layer.forward(apple_price, mandarin_price)
price = mul_tax_layer.forward(total_price, tax)

print(price)

715.0000000000001


코드가 다소 길어졌지만, 하나하나의 명령은 단순하다. 필요한 계층을 만들어 순전파 메소드인 forward()를 적절한 순서로 호출한다. 그런 다음 순전파와 반대 순서로 역전파 메소드인 backward()를 호출하면 원하는 미분이 나온다.

이처럼 계산 그래프에서의 계층은 쉽게 구현할 수 있으며, 이를 사용해 복잡한 미분도 계산할 수 있다. 다음 절에서는 신경마에서 사용하는 계층을 구현해보자.